In [100]:
import pandas as pd
import numpy as np
import os
from pathlib2 import Path
import re
import shutil
import ProjectDirectory as directory

# preprocess filings
import string
from nltk import word_tokenize
from nltk.stem import PorterStemmer

# to vectorize filing
from sklearn.feature_extraction.text import CountVectorizer


# This Python 3 environment comes with many helpful analytics libraries installed
#load packages
import sys #access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features
print("pandas version: {}". format(pd.__version__))

import matplotlib #collection of functions for scientific and publication-ready visualization
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import scipy as sp #collection of functions for scientific computing and advance mathematics
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
print("IPython version: {}". format(IPython.__version__)) 

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

#misc libraries
import random
import time


#ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Libraries for text

# !pip install PyDrive
# !pip install gensim
# !pip install pyldavis
# !python -m spacy download en
from nltk.corpus import stopwords
from nltk.util import ngrams

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from nltk.tokenize import TweetTokenizer
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split, cross_val_score

!pip install wordcloud
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier


from bs4 import BeautifulSoup
import requests as rq
import urllib.request as url
from bs4 import BeautifulSoup as bs

import pandas as pd
pd.set_option("display.max_colwidth", 200)
import numpy as np
import re
import spacy
import gzip
# spacy.load('en_core_web_sm')

import gensim
from gensim import corpora

Python version: 3.11.2 (main, Feb 16 2023, 02:51:42) [Clang 14.0.0 (clang-1400.0.29.202)]
pandas version: 2.2.3
matplotlib version: 3.8.3
NumPy version: 1.26.4
SciPy version: 1.11.1
IPython version: 8.14.0
scikit-learn version: 1.4.1.post1


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rishikumra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


# Preprocessing

**Import stopwords from LoughranMcDonald Master Dictionary**

In [2]:
def import_master_dict_stopwords(stopwords_file_path = os.path.join(directory.get_project_dir(), 'master-dict', 'StopWords_Generic.txt')):
#     os.chdir(stopwords_file_dir)
#     stopwords = pd.read_csv('StopWords_Generic.txt', header=None)
    stopwords = pd.read_csv(stopwords_file_path, header=None)[0].tolist()
    stopwords = frozenset([word.lower() for word in stopwords])
    return stopwords

In [3]:
def preprocess_filing(text, stopwords=True, stemming=False):
    
    # remove punctuations
    punctuation_list = set(string.punctuation)
    text = ''.join(word for word in text if word not in punctuation_list)
    
    tokens = word_tokenize(text)
    
    if stopwords:
        stopwords = import_master_dict_stopwords()
        tokens = [word for word in tokens if word not in stopwords]
        tokens = [word.lower() for word in tokens]

    if stemming:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(word) for word in tokens]
                
    return tokens

In [4]:
def vectorize_and_preprocess_filings(filings_list):
    """vectorizes and preprocesses filings for each company"""
    
    vectorizer = CountVectorizer(tokenizer=preprocess_filing)
    X = vectorizer.fit_transform(filings_list)
    return X

# Calculating Similarity

In [5]:
def calculate_consine_similarity(a, b):
    cos_sim = np.dot(a,b) / ( np.linalg.norm(a) * np.linalg.norm(b) )
    return cos_sim

In [455]:
word_dict = pd.read_csv('/Users/rishikumra/Downloads/Projects/sentiment-analysis-sec-master/data/dict.csv')
# len(list(word_dict[word_dict.Negative >0].Word))
# len(list(word_dict[word_dict.Positive >0].Word))

2345

In [440]:
from textblob import TextBlob

word_dict = pd.read_csv('/Users/rishikumra/Downloads/Projects/sentiment-analysis-sec-master/master-dict/LoughranMcDonald_MasterDictionary_2016.csv')
positive_txt = list(word_dict[word_dict.Positive >0].Word)
negative_txt = list(word_dict[word_dict.Negative >0].Word)
litigiuous_txt = list(word_dict[word_dict.Litigious >0].Word)
uncertainty_txt = list(word_dict[word_dict.Uncertainty >0].Word)
constaining_txt = list(word_dict[word_dict.Constraining >0].Word)
superfluous_txt = list(word_dict[word_dict.Superfluous >0].Word)
interesting_txt = list(word_dict[word_dict.Interesting >0].Word)
modal_txt = list(word_dict[word_dict.Modal >0].Word)
Irr_verb = list(word_dict[word_dict.Irr_Verb >0].Word)
Harvard_IV = list(word_dict[word_dict.Harvard_IV >0].Word)
# sentiments = ['Negative','Positive','Uncertainty','Litigious','Constraining','Superfluous',
#               'Interesting','Modal','Irr_Verb','Harvard_IV']

sent_list = [positive_txt, negative_txt, litigiuous_txt, uncertainty_txt, constaining_txt, superfluous_txt, interesting_txt, modal_txt, Irr_verb, Harvard_IV]
for k in sent_list:
    for i in range(len(k)):
        k[i] = k[i].lower()
#creating a dictionary of contractions 

contractions = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

#function for defining contraction's:

def expand_contractions(text):
    for word in text.split():
        if word.lower() in contractions:
            text = text.replace(word, contractions[word.lower()])
    return text

#function for removing unicode data :

import unicodedata
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

#function for removing all the scrub words
def scrub_words(text):
    #Replace \xao characters in text
    text = re.sub('\xa0', ' ', text)
    
    #Replace non ascii / not words and digits
    text = re.sub("(\\W|\\d)",' ',text)
    
    #Replace new line characters and following text until space
    text = re.sub('\n(\w*?)[\s]', '', text)
    
    #Remove html markup
    text = re.sub("<.*?>", ' ', text)
    
    #Remove extra spaces from the text
    text = re.sub("\s+", ' ', text)
    return text


#list of stop words from the observations by reading some level-1 cleaned documents 

stopwords = ['dtd','copyright','llc','en','html','fe','ed','webfilings','e','vk','g','zip code', 'pagebreak','html' 
             'w','c','en','table','body','par','value','per','securities','exchange','comission','telephone','number',
             'zip', 'code', 'end', 'page','xbrl','begin','dc','aa','aaa', 'aaa aa','ab','abn','abn amro','abnormal',
             'abo','abs','ac','az','ba','baa','aoci','aol','apb','api','app','ann','anne','amp','amt','anda','bla','bit',
             'bio','bhc','bb','bbb','bbl','bbls','bc','bcf','bcfe','apr','arc','aro','asa','asa','asc','asic','asp','asr',
             'asu','asus','ave','bms','bnp','bny','boe','blvd','bms','boe','bps','bs','btu', 'btus','ca','cad','cal','ccc',
             'cceeff','cdo','cdos','cds','ce','cede','cg','chk','cmsa','col','com','con','conway','ct','dd','de','dan',
             'dana','dea','wti','wto','wv','wyeth','wyoming','xannual','xerox','xi','xii','xiii','xindicate','xiv','xix','xl',
             'xthe','xv','xvi','xvii','xviii', 'xx','xxi','xxx','wi','vt','vs','von''vie','via','vi','var','ta','tab','tam',
             'td','tdr','tdrs','te','sur','ss','sr','sq','sp','sop','sip','sd','sdn','se',
             '__________________________________________ '
            '__________ ']

def review_to_words(raw_review):
    
    remove = re.sub(r'\b\w{1,3}\b', '', raw_review) #removing all words less than 4 characters 
    letters_only = re.sub("[^a-zA-Z]", " ", remove) 
    word = letters_only.lower().split()
  
    meaningful_words = [w for w in word if not w in stopwords] 
    return( " ".join(meaningful_words))
 
#function for defining contraction's:

def expand_contractions(text):
    for word in text.split():
        if word.lower() in contractions:
            text = text.replace(word, contractions[word.lower()])
    return text

#function for removing unicode data :

import unicodedata
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

#function for removing all the scrub words
def scrub_words(text):
    #Replace \xao characters in text
    text = re.sub('\xa0', ' ', text)
    
    #Replace non ascii / not words and digits
    text = re.sub("(\\W|\\d)",' ',text)
    
    #Replace new line characters and following text until space
    text = re.sub('\n(\w*?)[\s]', '', text)
    
    #Remove html markup
    text = re.sub("<.*?>", ' ', text)
    
    #Remove extra spaces from the text
    text = re.sub("\s+", ' ', text)
    return text


#Let's define functions for creating new columns 

# Generic stop wordss list from the mcdonald weblink\
from nltk.tokenize import RegexpTokenizer


stopWordsFile = '/Users/rishikumra/Downloads/Projects/sentiment-analysis-sec-master/StopWords_Generic.txt'
with open(stopWordsFile ,'r') as stop_words:
    stopWords = stop_words.read().lower()
stopWordList = stopWords.split('\n')
stopWordList[-1:] = []
# Tokenizer
def tokenizer(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    filtered_words = list(filter(lambda token: token not in stopWordList, tokens))
    return filtered_words

#===============================================================================
# Loading positive words
# positiveWordsFile = '/Users/rishikumra/Downloads/Projects/sentiment-analysis-sec-master/positive-words.txt'
# with open(positiveWordsFile,'r') as posfile:
#     positivewords=posfile.read().lower()
positiveWordList=positive_txt

# Calculating positive score 
def positive_score(text):
    numPosWords = 0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in positiveWordList:
            numPosWords  += 1
    
    sumPos = numPosWords
    return sumPos
#===============================================================================
# Loading negative words
# nagitiveWordsFile = '/Users/rishikumra/Downloads/Projects/sentiment-analysis-sec-master/negative-words.txt'
# with open(nagitiveWordsFile ,'r') as negfile:
#     negativeword=negfile.read().lower()

def analyze_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity  # Value ranges from -1 (negative) to 1 (positive)
    
    return sentiment


negativeWordList=negative_txt


# Calculating Negative score
def negative_word(text):
    numNegWords=0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in negativeWordList:
            numNegWords -=1
    sumNeg = numNegWords 
    sumNeg = sumNeg * -1
    return sumNeg
#===============================================================================
# Loading litigous words
# litigous = '/content/litiguous.txt'
# with open(litigous ,'r') as litfile:
#     litigousword=litfile.read().lower()
litigouswordlist =litigiuous_txt


# Calculating litigous score 
def litigous_score(text):
    numLitWords = 0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in litigouswordlist:
            numLitWords  += 1
    
    sumLit = numLitWords
    return sumLit
#===============================================================================	
# Loading interesting words
# interestingWordsfile = '/content/interestingwords.txt'
# with open(interestingWordsfile ,'r') as intfile:
#     interestingWord=intfile.read().lower()
interestingWordList=interesting_txt


# Calculating interestingword score 
def interesting_score(text):
    numintwords = 0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in interestingWordList:
            numintwords  += 1
    
    sumInt = numintwords
    return sumInt

#===============================================================================
# Loading superflous words
# superflousfile = '/content/superfluous.txt'
# with open(superflousfile ,'r') as supfile:
#     supword=supfile.read().lower()
supWordList=superfluous_txt


# Calculating sup_score 
def sup_score(text):
    numSupWords = 0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in supWordList:
            numSupWords  += 1

         
    sumSup = numSupWords
    return sumSup
#===============================================================================
# Loading Modalstrong words
# modalstrongfile = '/content/modal_strong.txt'
# with open(modalstrongfile ,'r') as modfile:
#     modstrongword=modfile.read().lower()
modstrongwordlist=modal_txt


# Calculating modal strongscore 
def modal_strong_store(text):
    nomodstr = 0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in modstrongwordlist:
            nomodstr  += 1
    
    summodstr = nomodstr
    return summodstr
#===============================================================================	
# # Loading Modalneutral words
# modalneutralfile = '/content/modal_neutral.txt'
# with open(modalneutralfile ,'r') as modn:
#     modneuword=modn.read().lower()
# modneuwordlist=modneuword.split('\n')


# # Calculating modal_neu_score
# def modal_neutral_score(text):
#     nomodneu = 0
#     rawToken = tokenizer(text)
#     for word in rawToken:
#         if word in modneuwordlist:
#             nomodneu  += 1
    
#     summodneu = nomodneu
#     return summodneu

# #===============================================================================
# # Loading modal_weak words
# modal_weak_file = '/content/modal_weak.txt'
# with open(modal_weak_file ,'r') as modweak:
#     modweakword=modweak.read().lower()
# modweaklist=modweakword.split('\n')


# # Calculating mod_weak_score 
# def mod_weak_score(text):
#     nomodweak = 0
#     rawToken = tokenizer(text)
#     for word in rawToken:
#         if word in modweaklist:
#             nomodweak  += 1
    
#     summodweak = nomodweak
#     return summodweak
	
#===============================================================================
# # Loading dictionary_countcount words
# # dictionary_wordfile = '/content/dictionary.txt'
# # with open(dictionary_wordfile ,'r') as dicfile:
# #     dicword=dicfile.read().lower()
# dicwordlist=


# # Calculating dic_word_score score 
# def dic_word_score(text):
#     nodicwords = 0
#     rawToken = tokenizer(text)
#     for word in rawToken:
#         if word in dicwordlist:
#             nodicwords  += 1
    
#     sumdic = nodickwords
#     return sumdic
#===============================================================================

# Loading harvard_1 words
# harvard_1_file = '/content/harvard_1.txt'
# with open(harvard_1_file ,'r') as harfile:
#     har_1_word=harfile.read().lower()
har_1_list=Harvard_IV


# Calculating dic_word_score score 
def har_1_score(text):
    nohar_1_words = 0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in har_1_list:
            nohar_1_words  += 1
    
    sumhar_1 = nohar_1_words
    return sumhar_1
#===============================================================================
# Calculating polarity score
def polarity_score(positiveScore, negativeScore):
    pol_score = (positiveScore - negativeScore) / ((positiveScore + negativeScore) + 0.000001)
    return pol_score

#===============================================================================
# Calculating Average sentence length 

'''# It will calculated using formula --- 
      Average Sentence Length = the number of words / the number of sentences'''
     
def average_sentence_length(text):
    sentence_list = sent_tokenize(text)
    tokens = tokenizer(text)
    totalWordCount = len(tokens)
    totalSentences = len(sentence_list)
    average_sent = 0
    if totalSentences != 0:
        average_sent = totalWordCount / totalSentences
    
    average_sent_length= average_sent
    
    return round(average_sent_length)
#===============================================================================
# Calculating percentage of complex words

''' It is calculated using Percentage of 
            Complex words = the number of complex words / the number of words''' 

def percentage_complex_word(text):
    tokens = tokenizer(text)
    complexWord = 0
    complex_word_percentage = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    if len(tokens) != 0:
        complex_word_percentage = complexWord/len(tokens)
    
    return complex_word_percentage
#===============================================================================

# calculating Fog Index 

'''# Fog index is calculated using --
    Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)'''

def fog_index(averageSentenceLength, percentageComplexWord):
    fogIndex = 0.4 * (averageSentenceLength + percentageComplexWord)
    return fogIndex

#===============================================================================
# Counting complex words
def complex_word_count(text):
    tokens = tokenizer(text)
    complexWord = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    return complexWord
#===============================================================================
#Counting total words

def total_word_count(text):
    tokens = tokenizer(text)
    return len(tokens)

#===============================================================================
# calculating uncertainty_score
# uncertainty_dictionaryFile= '/content/uncertainity.txt'
# with open(uncertainty_dictionaryFile ,'r') as uncertain_dict:
#     uncertainDict=uncertain_dict.read().lower()
uncertainDictionary = uncertainty_txt

#uncertainity score 
def uncertainty_score(text):
    uncertainWordnum =0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in uncertainDictionary:
            uncertainWordnum +=1
    sumUncertainityScore = uncertainWordnum 
    
    return sumUncertainityScore
#===============================================================================
# calculating constraining score
# constraining_dictionaryFile = '/content/constrained.txt'
# with open(constraining_dictionaryFile ,'r') as constraining_dict:
#     constrainDict=constraining_dict.read().lower()
constrainDictionary = constaining_txt


def constraining_score(text):
    constrainWordnum =0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in constrainDictionary:
            constrainWordnum +=1
    sumConstrainScore = constrainWordnum 
    
    return sumConstrainScore
#===============================================================================
# Calculating positive word proportion
def positive_word_prop(positiveScore,wordcount):
    positive_word_proportion = 0
    if wordcount !=0:
        positive_word_proportion = positiveScore / wordcount
        
    return positive_word_proportion

#===============================================================================
# Calculating negative word proportion

def negative_word_prop(negativeScore,wordcount):
    negative_word_proportion = 0
    if wordcount !=0:
        negative_word_proportion = negativeScore / wordcount
        
    return negative_word_proportion
#===============================================================================
# Calculating uncertain word proportion
def uncertain_word_prop(uncertainScore,wordcount):
    uncertain_word_proportion = 0
    if wordcount !=0:
        uncertain_word_proportion = uncertainScore / wordcount
        
    return uncertain_word_proportion

#===============================================================================
# Calculating uncertain word proportion
def uncertain_word_prop(uncertainScore,wordcount):
    uncertain_word_proportion = 0
    if wordcount !=0:
        uncertain_word_proportion = uncertainScore / wordcount
        
    return uncertain_word_proportion

#===============================================================================
# Calculating constraining word proportion
def constraining_word_prop(constrainingScore,wordcount):
    constraining_word_proportion = 0
    if wordcount !=0:
        constraining_word_proportion = constrainingScore / wordcount
        
    return constraining_word_proportion
#===============================================================================
# calculating Constraining words for whole report
def constrain_word_whole(mdaText):
    wholeDoc = mdaText
    constrainWordnumWhole =0
    rawToken = tokenizer(wholeDoc)
    for word in rawToken:
        if word in constrainDictionary:
            constrainWordnumWhole +=1
    sumConstrainScoreWhole = constrainWordnumWhole 
    
    return sumConstrainScoreWhole
#===============================================================================
  

In [463]:
final_df

,year,company,litigous_score,superfluous_score,interesting_score,modal_score,harvard_1,polarity_score,avg_sen_length,fog_index,Future Return
0,2014,AMERICAN EXPRESS CO,0.017580,0.000224,0.001607,0.014124,0.047509,0.053458,33202,13280.973893,0.036627
1,2015,AMERICAN EXPRESS CO,0.018550,0.000172,0.001603,0.014447,0.048322,0.051530,32329,12931.773862,-0.242095
2,2016,AMERICAN EXPRESS CO,0.013058,0.000177,0.001374,0.011426,0.053947,0.057735,46143,18457.367887,0.085822
3,2017,AMERICAN EXPRESS CO,0.013551,0.000149,0.001588,0.012558,0.057231,0.061721,37799,15119.770491,0.362164
4,2018,AMERICAN EXPRESS CO,0.012938,0.000081,0.001629,0.012615,0.058108,0.064072,38732,15492.970464,-0.026202
5,2019,AMERICAN EXPRESS CO,0.013071,0.000118,0.001368,0.013307,0.056927,0.065217,36912,14764.973992,0.325174
6,2014,Facebook Inc,0.010787,0.000109,0.001775,0.017205,0.042083,0.045054,21236,8494.584065,0.427630
7,2015,Facebook Inc,0.010144,0.000141,0.001803,0.017583,0.044043,0.044256,20623,8249.385967,0.341451
8,2016,Facebook Inc,0.010719,0.000114,0.001686,0.018550,0.045590,0.046100,20292,8116.984073,0.099274
9,2017,Facebook Inc,0.012631,0.000104,0.001641,0.019142,0.046903,0.047337,22332,8932.981426,0.533768


In [441]:
import nltk
nltk.download('punkt_tab')
final_df = pd.DataFrame(columns = ['year', 'company', 'positive_score', 'negative_score', 'litigous_score', 'superfluous_score', 'interesting_score', 'modal_score', 'harvard_1', 'polarity_score', 'avg_sen_length', 'fog_index'])
company_name_list = ['AMERICAN EXPRESS CO', 'MICROSOFT CORP', 'Facebook Inc', 'NETFLIX INC', 'AMAZON COM INC']
for comp_curr in company_name_list:
    company_dir = os.path.join(project_dir, 'sec-filings-downloaded', comp_curr)
    try:
        os.chdir(os.path.join(company_dir, 'cleaned_filings'))
    except:
        continue
    ten_k_dict = {}
    ten_q_dict = {}
    
    for file in os.listdir():
        if file.endswith('10-K'): 
            filing_year = int(file[8:12])
            ten_k_dict[filing_year] = file

    years_ = [2014,2015,2016,2017,2018,2019]
    for max_ten_k_year in years_:
        # try:
            with open(ten_k_dict[max_ten_k_year]) as file:
                latest_ten_k = file.readline()
            if latest_ten_k!=None:
                print(comp_curr, max_ten_k_year)
            #using contractions dictionary to make corrections 
                latest_ten_k = expand_contractions(re.sub('’', "'", latest_ten_k))

                #stripping the words using space 
                latest_ten_k = latest_ten_k.strip().lower()

                #removing accented characters 
                latest_ten_k = remove_accented_chars(latest_ten_k)

                #re-placing " " " with space 
                latest_ten_k = latest_ten_k.replace('"', '')

                # Removing url's from the text
                url_reg  = r'[a-z]*[:.]+\S+'
                latest_ten_k = re.sub(url_reg, '', latest_ten_k)

                latest_ten_k = re.sub(r"\b[a-zA-Z]\b", "", latest_ten_k)

                #removing scrub_words
                latest_ten_k = scrub_words(latest_ten_k)

                #replace spaaces more than one with single space 
                latest_ten_k = re.sub("\s+", ' ', latest_ten_k)

                #finding the length of the words in the data
                word_count = len(latest_ten_k.split(' '))
                        
                data = review_to_words(latest_ten_k)

                positive_scor = positive_score(latest_ten_k)/len(latest_ten_k.split(' '))
                negative_scor = negative_word(latest_ten_k)/len(latest_ten_k.split(' '))
                litigous_scor = litigous_score(latest_ten_k)/len(latest_ten_k.split(' '))
                superfluous_scor = sup_score(latest_ten_k)/len(latest_ten_k.split(' '))
                interesting_socr = interesting_score(latest_ten_k)/len(latest_ten_k.split(' '))
                modal_scor = modal_strong_store(latest_ten_k)/len(latest_ten_k.split(' '))
                harvard_1 = har_1_score(latest_ten_k)/len(latest_ten_k.split(' '))
                polarity_scor = polarity_score(positive_scor, negative_scor)
                polar_ = analyze_sentiment(latest_ten_k)
                avg_sen_leng = average_sentence_length(latest_ten_k)
                fog_inde = fog_index(avg_sen_leng, percentage_complex_word(latest_ten_k))

                new_row = pd.DataFrame({
                    'year': [max_ten_k_year],
                    'company': [comp_curr],
                    'positive_score': [positive_scor],
                    'negative_score': [negative_scor],
                    'litigous_score': [litigous_scor],
                    'superfluous_score': [superfluous_scor],
                    'interesting_score': [interesting_socr],
                    'modal_score': [modal_scor],
                    'harvard_1': [harvard_1],
                    'polarity_score': [polar_],
                    'avg_sen_length': [avg_sen_leng],
                    'fog_index': [fog_inde]
                })
                final_df = pd.concat([final_df, new_row], ignore_index=False)
                # df = final_df.append({'year': max_ten_k_year, 'company': comp_curr, 'positive_score': positive_score}, ignore_index=True)
                # df
        # except:
        #     print('error for', comp_curr)
            
final_df.index = range(len(final_df))

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/rishikumra/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


AMERICAN EXPRESS CO 2014
AMERICAN EXPRESS CO 2015
AMERICAN EXPRESS CO 2016
AMERICAN EXPRESS CO 2017
AMERICAN EXPRESS CO 2018
AMERICAN EXPRESS CO 2019
Facebook Inc 2014
Facebook Inc 2015
Facebook Inc 2016
Facebook Inc 2017
Facebook Inc 2018
Facebook Inc 2019
NETFLIX INC 2014
NETFLIX INC 2015
NETFLIX INC 2016
NETFLIX INC 2017
NETFLIX INC 2018
NETFLIX INC 2019
AMAZON COM INC 2014
AMAZON COM INC 2015
AMAZON COM INC 2016
AMAZON COM INC 2017
AMAZON COM INC 2018
AMAZON COM INC 2019


In [456]:
final_df = final_df.drop(columns=['positive_score', 'negative_score'])

In [373]:
# # # # company_dir = os.path.join(project_dir, 'sec-filings-downloaded', 'AMERICAN EXPRESS CO')
# # # # os.chdir(os.path.join(company_dir, 'cleaned_filings'))

# # # # ten_k_dict = {}
# # # # ten_q_dict = {}

# # # # for file in os.listdir():
# # # #     if file.endswith('10-K'): 
# # # #         filing_year = int(file[8:12])
# # # #         ten_k_dict[filing_year] = file

# # # # ten_k_dict
# # # # import nltk
# # # # nltk.download('punkt_tab')
# # # # max_ten_k_year = max(ten_k_dict, key=ten_k_dict.get)
# # # # year_before_max_ten_k = max_ten_k_year - 1
# # # # max_ten_k_year = 2015

# # # # with open(ten_k_dict[max_ten_k_year]) as file:
# # # #     latest_ten_k = file.readline()
# # # # with open(ten_k_dict[year_before_max_ten_k]) as file:
# # # #     previous_ten_k = file.readline()

# # # # ten_k_vec = vectorize_and_preprocess_filings([latest_ten_k, previous_ten_k])
# # # # cosine_sim_ten_k = calculate_consine_similarity(ten_k_vec.toarray()[0], ten_k_vec.toarray()[1])
# # # # new_row = {
# # # #     'company': company, 
# # # #     'cosine_similarity': cosine_sim_ten_k,
# # # #     'latest_filing_dt': ten_k_dict[max_ten_k_year][8:18],
# # # #     'previous_filing_dt': ten_k_dict[year_before_max_ten_k][8:18]
# # # # }
# # # # df_ten_k_results = pd.concat([df_ten_k_results, pd.DataFrame([new_row])], ignore_index=True)

# # # # with open(ten_k_dict[max_ten_k_year]) as file:
# # # #     latest_ten_k = file.readline()
# # # # word_dict = pd.read_csv('/Users/rishikumra/Downloads/Projects/sentiment-analysis-sec-master/master-dict/LoughranMcDonald_MasterDictionary_2016.csv')

# # # # positive_txt = list(word_dict[word_dict.Positive >0].Word)
# # # # negative_txt = list(word_dict[word_dict.Negative >0].Word)
# # # # litigiuous_txt = list(word_dict[word_dict.Litigious >0].Word)
# # # # uncertainty_txt = list(word_dict[word_dict.Uncertainty >0].Word)
# # # # constaining_txt = list(word_dict[word_dict.Constraining >0].Word)
# # # # superfluous_txt = list(word_dict[word_dict.Superfluous >0].Word)
# # # # interesting_txt = list(word_dict[word_dict.Interesting >0].Word)
# # # # modal_txt = list(word_dict[word_dict.Modal >0].Word)
# # # # Irr_verb = list(word_dict[word_dict.Irr_Verb >0].Word)
# # # # Harvard_IV = list(word_dict[word_dict.Harvard_IV >0].Word)
# # # # # sentiments = ['Negative','Positive','Uncertainty','Litigious','Constraining','Superfluous',
# # # # #               'Interesting','Modal','Irr_Verb','Harvard_IV']

# # # # sent_list = [positive_txt, negative_txt, litigiuous_txt, uncertainty_txt, constaining_txt, superfluous_txt, interesting_txt, modal_txt, Irr_verb, Harvard_IV]

# # # word_dict
# # # sentiments = ['Negative','Positive','Uncertainty','Litigious','Constraining','Superfluous',
# # #               'Interesting','Modal','Irr_Verb','Harvard_IV']
# # # #creating a dictionary of contractions 

# # # contractions = {
# # # "ain't": "is not",
# # # "aren't": "are not",
# # # "can't": "cannot",
# # # "can't've": "cannot have",
# # # "'cause": "because",
# # # "could've": "could have",
# # # "couldn't": "could not",
# # # "couldn't've": "could not have",
# # # "didn't": "did not",
# # # "doesn't": "does not",
# # # "don't": "do not",
# # # "hadn't": "had not",
# # # "hadn't've": "had not have",
# # # "hasn't": "has not",
# # # "haven't": "have not",
# # # "he'd": "he would",
# # # "he'd've": "he would have",
# # # "he'll": "he will",
# # # "he'll've": "he he will have",
# # # "he's": "he is",
# # # "how'd": "how did",
# # # "how'd'y": "how do you",
# # # "how'll": "how will",
# # # "how's": "how is",
# # # "I'd": "I would",
# # # "I'd've": "I would have",
# # # "I'll": "I will",
# # # "I'll've": "I will have",
# # # "I'm": "I am",
# # # "I've": "I have",
# # # "i'd": "i would",
# # # "i'd've": "i would have",
# # # "i'll": "i will",
# # # "i'll've": "i will have",
# # # "i'm": "i am",
# # # "i've": "i have",
# # # "isn't": "is not",
# # # "it'd": "it would",
# # # "it'd've": "it would have",
# # # "it'll": "it will",
# # # "it'll've": "it will have",
# # # "it's": "it is",
# # # "let's": "let us",
# # # "ma'am": "madam",
# # # "mayn't": "may not",
# # # "might've": "might have",
# # # "mightn't": "might not",
# # # "mightn't've": "might not have",
# # # "must've": "must have",
# # # "mustn't": "must not",
# # # "mustn't've": "must not have",
# # # "needn't": "need not",
# # # "needn't've": "need not have",
# # # "o'clock": "of the clock",
# # # "oughtn't": "ought not",
# # # "oughtn't've": "ought not have",
# # # "shan't": "shall not",
# # # "sha'n't": "shall not",
# # # "shan't've": "shall not have",
# # # "she'd": "she would",
# # # "she'd've": "she would have",
# # # "she'll": "she will",
# # # "she'll've": "she will have",
# # # "she's": "she is",
# # # "should've": "should have",
# # # "shouldn't": "should not",
# # # "shouldn't've": "should not have",
# # # "so've": "so have",
# # # "so's": "so as",
# # # "that'd": "that would",
# # # "that'd've": "that would have",
# # # "that's": "that is",
# # # "there'd": "there would",
# # # "there'd've": "there would have",
# # # "there's": "there is",
# # # "they'd": "they would",
# # # "they'd've": "they would have",
# # # "they'll": "they will",
# # # "they'll've": "they will have",
# # # "they're": "they are",
# # # "they've": "they have",
# # # "to've": "to have",
# # # "wasn't": "was not",
# # # "we'd": "we would",
# # # "we'd've": "we would have",
# # # "we'll": "we will",
# # # "we'll've": "we will have",
# # # "we're": "we are",
# # # "we've": "we have",
# # # "weren't": "were not",
# # # "what'll": "what will",
# # # "what'll've": "what will have",
# # # "what're": "what are",
# # # "what's": "what is",
# # # "what've": "what have",
# # # "when's": "when is",
# # # "when've": "when have",
# # # "where'd": "where did",
# # # "where's": "where is",
# # # "where've": "where have",
# # # "who'll": "who will",
# # # "who'll've": "who will have",
# # # "who's": "who is",
# # # "who've": "who have",
# # # "why's": "why is",
# # # "why've": "why have",
# # # "will've": "will have",
# # # "won't": "will not",
# # # "won't've": "will not have",
# # # "would've": "would have",
# # # "wouldn't": "would not",
# # # "wouldn't've": "would not have",
# # # "y'all": "you all",
# # # "y'all'd": "you all would",
# # # "y'all'd've": "you all would have",
# # # "y'all're": "you all are",
# # # "y'all've": "you all have",
# # # "you'd": "you would",
# # # "you'd've": "you would have",
# # # "you'll": "you will",
# # # "you'll've": "you will have",
# # # "you're": "you are",
# # # "you've": "you have"
# # # }
# # #function for defining contraction's:

# # def expand_contractions(text):
# #     for word in text.split():
# #         if word.lower() in contractions:
# #             text = text.replace(word, contractions[word.lower()])
# #     return text

# # #function for removing unicode data :

# # import unicodedata
# # def remove_accented_chars(text):
# #     text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
# #     return text

# # #function for removing all the scrub words
# # def scrub_words(text):
# #     #Replace \xao characters in text
# #     text = re.sub('\xa0', ' ', text)
    
# #     #Replace non ascii / not words and digits
# #     text = re.sub("(\\W|\\d)",' ',text)
    
# #     #Replace new line characters and following text until space
# #     text = re.sub('\n(\w*?)[\s]', '', text)
    
# #     #Remove html markup
# #     text = re.sub("<.*?>", ' ', text)
    
# #     #Remove extra spaces from the text
# #     text = re.sub("\s+", ' ', text)
# #     return text


# # #Let's define functions for creating new columns 

# # # Generic stop wordss list from the mcdonald weblink\
# # from nltk.tokenize import RegexpTokenizer


# # stopWordsFile = '/Users/rishikumra/Downloads/Projects/sentiment-analysis-sec-master/StopWords_Generic.txt'
# # with open(stopWordsFile ,'r') as stop_words:
# #     stopWords = stop_words.read().lower()
# # stopWordList = stopWords.split('\n')
# # stopWordList[-1:] = []
# # # Tokenizer
# # def tokenizer(text):
# #     text = text.lower()
# #     tokenizer = RegexpTokenizer(r'\w+')
# #     tokens = tokenizer.tokenize(text)
# #     filtered_words = list(filter(lambda token: token not in stopWordList, tokens))
# #     return filtered_words

# # #===============================================================================
# # # Loading positive words
# # positiveWordList=positive_txt

# # # Calculating positive score 
# # def positive_score(text):
# #     numPosWords = 0
# #     rawToken = tokenizer(text)
# #     for word in rawToken:
# #         if word in positiveWordList:
# #             numPosWords  += 1
    
# #     sumPos = numPosWords
# #     return sumPos
# # #===============================================================================
# # # Loading negative words
# # # nagitiveWordsFile = '/Users/rishikumra/Downloads/Projects/sentiment-analysis-sec-master/negative-words.txt'
# # # with open(nagitiveWordsFile ,'r') as negfile:
# # #     negativeword=negfile.read().lower()
# # negativeWordList=negative_txt


# # # Calculating Negative score
# # def negative_word(text):
# #     numNegWords=0
# #     rawToken = tokenizer(text)
# #     for word in rawToken:
# #         if word in negativeWordList:
# #             numNegWords -=1
# #     sumNeg = numNegWords 
# #     sumNeg = sumNeg * -1
# #     return sumNeg
# # #===============================================================================
# # # Loading litigous words
# # # litigous = '/content/litiguous.txt'
# # # with open(litigous ,'r') as litfile:
# # #     litigousword=litfile.read().lower()
# # litigouswordlist =litigiuous_txt


# # # Calculating litigous score 
# # def litigous_score(text):
# #     numLitWords = 0
# #     rawToken = tokenizer(text)
# #     for word in rawToken:
# #         if word in litigouswordlist:
# #             numLitWords  += 1
    
# #     sumLit = numLitWords
# #     return sumLit
# # #===============================================================================	
# # # Loading interesting words
# # # interestingWordsfile = '/content/interestingwords.txt'
# # # with open(interestingWordsfile ,'r') as intfile:
# # #     interestingWord=intfile.read().lower()
# # interestingWordList=interesting_txt


# # # Calculating interestingword score 
# # def interesting_score(text):
# #     numintwords = 0
# #     rawToken = tokenizer(text)
# #     for word in rawToken:
# #         if word in interestingWordList:
# #             numintwords  += 1
    
# #     sumInt = numintwords
# #     return sumInt

# # #===============================================================================
# # # Loading superflous words
# # # superflousfile = '/content/superfluous.txt'
# # # with open(superflousfile ,'r') as supfile:
# # #     supword=supfile.read().lower()
# # supWordList=superfluous_txt


# # # Calculating sup_score 
# # def sup_score(text):
# #     numSupWords = 0
# #     rawToken = tokenizer(text)
# #     for word in rawToken:
# #         if word in supWordList:
# #             numSupWords  += 1

         
# #     sumSup = numSupWords
# #     return sumSup
# # #===============================================================================
# # # Loading Modalstrong words
# # # modalstrongfile = '/content/modal_strong.txt'
# # # with open(modalstrongfile ,'r') as modfile:
# # #     modstrongword=modfile.read().lower()
# # modstrongwordlist=modal_txt


# # # Calculating modal strongscore 
# # def modal_strong_store(text):
# #     nomodstr = 0
# #     rawToken = tokenizer(text)
# #     for word in rawToken:
# #         if word in modstrongwordlist:
# #             nomodstr  += 1
    
# #     summodstr = nomodstr
# #     return summodstr
# # #===============================================================================	
# # # # Loading Modalneutral words
# # # modalneutralfile = '/content/modal_neutral.txt'
# # # with open(modalneutralfile ,'r') as modn:
# # #     modneuword=modn.read().lower()
# # # modneuwordlist=modneuword.split('\n')


# # # # Calculating modal_neu_score
# # # def modal_neutral_score(text):
# # #     nomodneu = 0
# # #     rawToken = tokenizer(text)
# # #     for word in rawToken:
# # #         if word in modneuwordlist:
# # #             nomodneu  += 1
    
# # #     summodneu = nomodneu
# # #     return summodneu

# # # #===============================================================================
# # # # Loading modal_weak words
# # # modal_weak_file = '/content/modal_weak.txt'
# # # with open(modal_weak_file ,'r') as modweak:
# # #     modweakword=modweak.read().lower()
# # # modweaklist=modweakword.split('\n')


# # # # Calculating mod_weak_score 
# # # def mod_weak_score(text):
# # #     nomodweak = 0
# # #     rawToken = tokenizer(text)
# # #     for word in rawToken:
# # #         if word in modweaklist:
# # #             nomodweak  += 1
    
# # #     summodweak = nomodweak
# # #     return summodweak
	
# # #===============================================================================
# # # # Loading dictionary_countcount words
# # # # dictionary_wordfile = '/content/dictionary.txt'
# # # # with open(dictionary_wordfile ,'r') as dicfile:
# # # #     dicword=dicfile.read().lower()
# # # dicwordlist=


# # # # Calculating dic_word_score score 
# # # def dic_word_score(text):
# # #     nodicwords = 0
# # #     rawToken = tokenizer(text)
# # #     for word in rawToken:
# # #         if word in dicwordlist:
# # #             nodicwords  += 1
    
# # #     sumdic = nodickwords
# # #     return sumdic
# # #===============================================================================

# # # Loading harvard_1 words
# # # harvard_1_file = '/content/harvard_1.txt'
# # # with open(harvard_1_file ,'r') as harfile:
# # #     har_1_word=harfile.read().lower()
# # har_1_list=Harvard_IV


# # # Calculating dic_word_score score 
# # def har_1_score(text):
# #     nohar_1_words = 0
# #     rawToken = tokenizer(text)
# #     for word in rawToken:
# #         if word in har_1_list:
# #             nohar_1_words  += 1
    
# #     sumhar_1 = nohar_1_words
# #     return sumhar_1
# # #===============================================================================
# # # Calculating polarity score
# # def polarity_score(positiveScore, negativeScore):
# #     pol_score = (positiveScore - negativeScore) / ((positiveScore + negativeScore) + 0.000001)
# #     return pol_score

# # #===============================================================================
# # # Calculating Average sentence length 

# # '''# It will calculated using formula --- 
# #       Average Sentence Length = the number of words / the number of sentences'''
     
# # def average_sentence_length(text):
# #     sentence_list = sent_tokenize(text)
# #     tokens = tokenizer(text)
# #     totalWordCount = len(tokens)
# #     totalSentences = len(sentence_list)
# #     average_sent = 0
# #     if totalSentences != 0:
# #         average_sent = totalWordCount / totalSentences
    
# #     average_sent_length= average_sent
    
# #     return round(average_sent_length)
# # #===============================================================================
# # # Calculating percentage of complex words

# # ''' It is calculated using Percentage of 
# #             Complex words = the number of complex words / the number of words''' 

# # def percentage_complex_word(text):
# #     tokens = tokenizer(text)
# #     complexWord = 0
# #     complex_word_percentage = 0
    
# #     for word in tokens:
# #         vowels=0
# #         if word.endswith(('es','ed')):
# #             pass
# #         else:
# #             for w in word:
# #                 if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
# #                     vowels += 1
# #             if(vowels > 2):
# #                 complexWord += 1
# #     if len(tokens) != 0:
# #         complex_word_percentage = complexWord/len(tokens)
    
# #     return complex_word_percentage
# # #===============================================================================

# # # calculating Fog Index 

# # '''# Fog index is calculated using --
# #     Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)'''

# # def fog_index(averageSentenceLength, percentageComplexWord):
# #     fogIndex = 0.4 * (averageSentenceLength + percentageComplexWord)
# #     return fogIndex

# # #===============================================================================
# # # Counting complex words
# # def complex_word_count(text):
# #     tokens = tokenizer(text)
# #     complexWord = 0
    
# #     for word in tokens:
# #         vowels=0
# #         if word.endswith(('es','ed')):
# #             pass
# #         else:
# #             for w in word:
# #                 if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
# #                     vowels += 1
# #             if(vowels > 2):
# #                 complexWord += 1
# #     return complexWord
# # #===============================================================================
# # #Counting total words

# # def total_word_count(text):
# #     tokens = tokenizer(text)
# #     return len(tokens)

# # #===============================================================================
# # # calculating uncertainty_score
# # # uncertainty_dictionaryFile= '/content/uncertainity.txt'
# # # with open(uncertainty_dictionaryFile ,'r') as uncertain_dict:
# # #     uncertainDict=uncertain_dict.read().lower()
# # uncertainDictionary = uncertainty_txt

# # #uncertainity score 
# # def uncertainty_score(text):
# #     uncertainWordnum =0
# #     rawToken = tokenizer(text)
# #     for word in rawToken:
# #         if word in uncertainDictionary:
# #             uncertainWordnum +=1
# #     sumUncertainityScore = uncertainWordnum 
    
# #     return sumUncertainityScore
# # #===============================================================================
# # # calculating constraining score
# # # constraining_dictionaryFile = '/content/constrained.txt'
# # # with open(constraining_dictionaryFile ,'r') as constraining_dict:
# # #     constrainDict=constraining_dict.read().lower()
# # constrainDictionary = constaining_txt


# # def constraining_score(text):
# #     constrainWordnum =0
# #     rawToken = tokenizer(text)
# #     for word in rawToken:
# #         if word in constrainDictionary:
# #             constrainWordnum +=1
# #     sumConstrainScore = constrainWordnum 
    
# #     return sumConstrainScore
# # #===============================================================================
# # # Calculating positive word proportion
# # def positive_word_prop(positiveScore,wordcount):
# #     positive_word_proportion = 0
# #     if wordcount !=0:
# #         positive_word_proportion = positiveScore / wordcount
        
# #     return positive_word_proportion

# # #===============================================================================
# # # Calculating negative word proportion

# # def negative_word_prop(negativeScore,wordcount):
# #     negative_word_proportion = 0
# #     if wordcount !=0:
# #         negative_word_proportion = negativeScore / wordcount
        
# #     return negative_word_proportion
# # #===============================================================================
# # # Calculating uncertain word proportion
# # def uncertain_word_prop(uncertainScore,wordcount):
# #     uncertain_word_proportion = 0
# #     if wordcount !=0:
# #         uncertain_word_proportion = uncertainScore / wordcount
        
# #     return uncertain_word_proportion

# # #===============================================================================
# # # Calculating uncertain word proportion
# # def uncertain_word_prop(uncertainScore,wordcount):
# #     uncertain_word_proportion = 0
# #     if wordcount !=0:
# #         uncertain_word_proportion = uncertainScore / wordcount
        
# #     return uncertain_word_proportion

# # #===============================================================================
# # # Calculating constraining word proportion
# # def constraining_word_prop(constrainingScore,wordcount):
# #     constraining_word_proportion = 0
# #     if wordcount !=0:
# #         constraining_word_proportion = constrainingScore / wordcount
        
# #     return constraining_word_proportion
# # #===============================================================================
# # # calculating Constraining words for whole report
# # def constrain_word_whole(mdaText):
# #     wholeDoc = mdaText
# #     constrainWordnumWhole =0
# #     rawToken = tokenizer(wholeDoc)
# #     for word in rawToken:
# #         if word in constrainDictionary:
# #             constrainWordnumWhole +=1
# #     sumConstrainScoreWhole = constrainWordnumWhole 
    
# #     return sumConstrainScoreWhole
# # #===============================================================================
  

# #using contractions dictionary to make corrections 
# latest_ten_k = expand_contractions(re.sub('’', "'", latest_ten_k))

# #stripping the words using space 
# latest_ten_k = latest_ten_k.strip().lower()

# #removing accented characters 
# latest_ten_k = remove_accented_chars(latest_ten_k)

# #re-placing " " " with space 
# latest_ten_k = latest_ten_k.replace('"', '')

# # Removing url's from the text
# url_reg  = r'[a-z]*[:.]+\S+'
# latest_ten_k = re.sub(url_reg, '', latest_ten_k)

# latest_ten_k = re.sub(r"\b[a-zA-Z]\b", "", latest_ten_k)

# #removing scrub_words
# latest_ten_k = scrub_words(latest_ten_k)

# #replace spaaces more than one with single space 
# latest_ten_k = re.sub("\s+", ' ', latest_ten_k)

# #finding the length of the words in the data
# word_count = len(latest_ten_k.split(' '))
# #list of stop words from the observations by reading some level-1 cleaned documents 

# stopwords = ['dtd','copyright','llc','en','html','fe','ed','webfilings','e','vk','g','zip code', 'pagebreak','html' 
#              'w','c','en','table','body','par','value','per','securities','exchange','comission','telephone','number',
#              'zip', 'code', 'end', 'page','xbrl','begin','dc','aa','aaa', 'aaa aa','ab','abn','abn amro','abnormal',
#              'abo','abs','ac','az','ba','baa','aoci','aol','apb','api','app','ann','anne','amp','amt','anda','bla','bit',
#              'bio','bhc','bb','bbb','bbl','bbls','bc','bcf','bcfe','apr','arc','aro','asa','asa','asc','asic','asp','asr',
#              'asu','asus','ave','bms','bnp','bny','boe','blvd','bms','boe','bps','bs','btu', 'btus','ca','cad','cal','ccc',
#              'cceeff','cdo','cdos','cds','ce','cede','cg','chk','cmsa','col','com','con','conway','ct','dd','de','dan',
#              'dana','dea','wti','wto','wv','wyeth','wyoming','xannual','xerox','xi','xii','xiii','xindicate','xiv','xix','xl',
#              'xthe','xv','xvi','xvii','xviii', 'xx','xxi','xxx','wi','vt','vs','von''vie','via','vi','var','ta','tab','tam',
#              'td','tdr','tdrs','te','sur','ss','sr','sq','sp','sop','sip','sd','sdn','se',
#              '__________________________________________ '
#             '__________ ']
# def review_to_words(raw_review):
    
#     remove = re.sub(r'\b\w{1,3}\b', '', raw_review) #removing all words less than 4 characters 
#     letters_only = re.sub("[^a-zA-Z]", " ", remove) 
#     word = letters_only.lower().split()
  
#     meaningful_words = [w for w in word if not w in stopwords] 
#     return( " ".join(meaningful_words))
 
# data = review_to_words(latest_ten_k)
 
# for k in sent_list:
#     for i in range(len(k)):
#         k[i] = k[i].lower()
# #Let's define functions for creating new columns 

# # Generic stop wordss list from the mcdonald weblink\
# from nltk.tokenize import RegexpTokenizer


# stopWordsFile = '/Users/rishikumra/Downloads/Projects/sentiment-analysis-sec-master/StopWords_Generic.txt'
# with open(stopWordsFile ,'r') as stop_words:
#     stopWords = stop_words.read().lower()
# stopWordList = stopWords.split('\n')
# stopWordList[-1:] = []
# # Tokenizer
# def tokenizer(text):
#     text = text.lower()
#     tokenizer = RegexpTokenizer(r'\w+')
#     tokens = tokenizer.tokenize(text)
#     filtered_words = list(filter(lambda token: token not in stopWordList, tokens))
#     return filtered_words

# #===============================================================================
# # Loading positive words
# positiveWordList=positive_txt

# # Calculating positive score 
# def positive_score(text):
#     numPosWords = 0
#     rawToken = tokenizer(text)
#     for word in rawToken:
#         if word in positiveWordList:
#             numPosWords  += 1
    
#     sumPos = numPosWords
#     return sumPos
# #===============================================================================
# # Loading negative words

# negativeWordList=negative_txt


# # Calculating Negative score
# def negative_word(text):
#     numNegWords=0
#     rawToken = tokenizer(text)
#     for word in rawToken:
#         if word in negativeWordList:
#             numNegWords -=1
#     sumNeg = numNegWords 
#     sumNeg = sumNeg * -1
#     return sumNeg
# #===============================================================================
# # Loading litigous words

# litigouswordlist =litigiuous_txt


# # Calculating litigous score 
# def litigous_score(text):
#     numLitWords = 0
#     rawToken = tokenizer(text)
#     for word in rawToken:
#         if word in litigouswordlist:
#             numLitWords  += 1
    
#     sumLit = numLitWords
#     return sumLit
# #===============================================================================	
# # Loading interesting words
# interestingWordList=interesting_txt


# # Calculating interestingword score 
# def interesting_score(text):
#     numintwords = 0
#     rawToken = tokenizer(text)
#     for word in rawToken:
#         if word in interestingWordList:
#             numintwords  += 1
    
#     sumInt = numintwords
#     return sumInt

# #===============================================================================
# # Loading superflous words
# supWordList=superfluous_txt


# # Calculating sup_score 
# def sup_score(text):
#     numSupWords = 0
#     rawToken = tokenizer(text)
#     for word in rawToken:
#         if word in supWordList:
#             numSupWords  += 1

         
#     sumSup = numSupWords
#     return sumSup
# #===============================================================================
# # Loading Modalstrong words
# modstrongwordlist=modal_txt


# # Calculating modal strongscore 
# def modal_strong_store(text):
#     nomodstr = 0
#     rawToken = tokenizer(text)
#     for word in rawToken:
#         if word in modstrongwordlist:
#             nomodstr  += 1
    
#     summodstr = nomodstr
#     return summodstr
# #===============================================================================	
# har_1_list=Harvard_IV


# # Calculating dic_word_score score 
# def har_1_score(text):
#     nohar_1_words = 0
#     rawToken = tokenizer(text)
#     for word in rawToken:
#         if word in har_1_list:
#             nohar_1_words  += 1
    
#     sumhar_1 = nohar_1_words
#     return sumhar_1
# #===============================================================================
# # Calculating polarity score
# def polarity_score(positiveScore, negativeScore):
#     pol_score = (positiveScore - negativeScore) / ((positiveScore + negativeScore) + 0.000001)
#     return pol_score

# #===============================================================================
# # Calculating Average sentence length 

# '''# It will calculated using formula --- 
#       Average Sentence Length = the number of words / the number of sentences'''
     
# def average_sentence_length(text):
#     sentence_list = sent_tokenize(text)
#     tokens = tokenizer(text)
#     totalWordCount = len(tokens)
#     totalSentences = len(sentence_list)
#     average_sent = 0
#     if totalSentences != 0:
#         average_sent = totalWordCount / totalSentences
    
#     average_sent_length= average_sent
    
#     return round(average_sent_length)
# #===============================================================================
# # Calculating percentage of complex words

# ''' It is calculated using Percentage of 
#             Complex words = the number of complex words / the number of words''' 

# def percentage_complex_word(text):
#     tokens = tokenizer(text)
#     complexWord = 0
#     complex_word_percentage = 0
    
#     for word in tokens:
#         vowels=0
#         if word.endswith(('es','ed')):
#             pass
#         else:
#             for w in word:
#                 if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
#                     vowels += 1
#             if(vowels > 2):
#                 complexWord += 1
#     if len(tokens) != 0:
#         complex_word_percentage = complexWord/len(tokens)
    
#     return complex_word_percentage
# #===============================================================================

# # calculating Fog Index 

# '''# Fog index is calculated using --
#     Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)'''

# def fog_index(averageSentenceLength, percentageComplexWord):
#     fogIndex = 0.4 * (averageSentenceLength + percentageComplexWord)
#     return fogIndex

# #===============================================================================
# # Counting complex words
# def complex_word_count(text):
#     tokens = tokenizer(text)
#     complexWord = 0
    
#     for word in tokens:
#         vowels=0
#         if word.endswith(('es','ed')):
#             pass
#         else:
#             for w in word:
#                 if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
#                     vowels += 1
#             if(vowels > 2):
#                 complexWord += 1
#     return complexWord
# #===============================================================================
# #Counting total words

# def total_word_count(text):
#     tokens = tokenizer(text)
#     return len(tokens)

# #===============================================================================
# # calculating uncertainty_score
# # uncertainty_dictionaryFile= '/content/uncertainity.txt'
# # with open(uncertainty_dictionaryFile ,'r') as uncertain_dict:
# #     uncertainDict=uncertain_dict.read().lower()
# uncertainDictionary = uncertainty_txt

# #uncertainity score 
# def uncertainty_score(text):
#     uncertainWordnum =0
#     rawToken = tokenizer(text)
#     for word in rawToken:
#         if word in uncertainDictionary:
#             uncertainWordnum +=1
#     sumUncertainityScore = uncertainWordnum 
    
#     return sumUncertainityScore
# #===============================================================================
# # calculating constraining score
# # constraining_dictionaryFile = '/content/constrained.txt'
# # with open(constraining_dictionaryFile ,'r') as constraining_dict:
# #     constrainDict=constraining_dict.read().lower()
# constrainDictionary = constaining_txt


# def constraining_score(text):
#     constrainWordnum =0
#     rawToken = tokenizer(text)
#     for word in rawToken:
#         if word in constrainDictionary:
#             constrainWordnum +=1
#     sumConstrainScore = constrainWordnum 
    
#     return sumConstrainScore
# #===============================================================================
# # Calculating positive word proportion
# def positive_word_prop(positiveScore,wordcount):
#     positive_word_proportion = 0
#     if wordcount !=0:
#         positive_word_proportion = positiveScore / wordcount
        
#     return positive_word_proportion

# #===============================================================================
# # Calculating negative word proportion

# def negative_word_prop(negativeScore,wordcount):
#     negative_word_proportion = 0
#     if wordcount !=0:
#         negative_word_proportion = negativeScore / wordcount
        
#     return negative_word_proportion
# #===============================================================================
# # Calculating uncertain word proportion
# def uncertain_word_prop(uncertainScore,wordcount):
#     uncertain_word_proportion = 0
#     if wordcount !=0:
#         uncertain_word_proportion = uncertainScore / wordcount
        
#     return uncertain_word_proportion

# #===============================================================================
# # Calculating uncertain word proportion
# def uncertain_word_prop(uncertainScore,wordcount):
#     uncertain_word_proportion = 0
#     if wordcount !=0:
#         uncertain_word_proportion = uncertainScore / wordcount
        
#     return uncertain_word_proportion

# #===============================================================================
# # Calculating constraining word proportion
# def constraining_word_prop(constrainingScore,wordcount):
#     constraining_word_proportion = 0
#     if wordcount !=0:
#         constraining_word_proportion = constrainingScore / wordcount
        
#     return constraining_word_proportion
# #===============================================================================
# # calculating Constraining words for whole report
# def constrain_word_whole(mdaText):
#     wholeDoc = mdaText
#     constrainWordnumWhole =0
#     rawToken = tokenizer(wholeDoc)
#     for word in rawToken:
#         if word in constrainDictionary:
#             constrainWordnumWhole +=1
#     sumConstrainScoreWhole = constrainWordnumWhole 
    
#     return sumConstrainScoreWhole
# #===============================================================================
  
# # final_df = pd.DataFrame(columns = ['year', 'company', 'positive_score', 'negative_score', 'litigous_score', 'superfluous_score', 'interesting_score', 'modal_score', 'harvard_1', 'polarity_score', 'avg_sen_length', 'fog_index'])

# positive_score = positive_score(latest_ten_k)/len(latest_ten_k.split(' '))
# negative_score = negative_word(latest_ten_k)/len(latest_ten_k.split(' '))
# litigous_score = litigous_score(latest_ten_k)/len(latest_ten_k.split(' '))
# superfluous_score = sup_score(latest_ten_k)/len(latest_ten_k.split(' '))
# interesting_socre = interesting_score(latest_ten_k)/len(latest_ten_k.split(' '))
# modal_score = modal_strong_store(latest_ten_k)/len(latest_ten_k.split(' '))
# harvard_1 = har_1_score(latest_ten_k)/len(latest_ten_k.split(' '))
# polarity_score = polarity_score(positive_score, negative_score)
# avg_sen_length = average_sentence_length(latest_ten_k)
# fog_index = fog_index(avg_sen_length, percentage_complex_word(latest_ten_k))

# new_row = pd.DataFrame({
#     'year': [max_ten_k_year],
#     'company': [comp_curr],
#     'positive_score': [positive_score],
#     'negative_score': [negative_score],
#     'litigous_score': [litigous_score],
#     'superfluous_score': [superfluous_score],
#     'interesting_score': [interesting_socre],
#     'modal_score': [modal_score],
#     'harvard_1': [harvard_1],
#     'polarity_score': [polarity_score],
#     'avg_sen_length': [avg_sen_length],
#     'fog_index': [fog_index]
# })
# final_df = pd.concat([final_df, new_row], ignore_index=False)
# # df = final_df.append({'year': max_ten_k_year, 'company': comp_curr, 'positive_score': positive_score}, ignore_index=True)
# final_df
# # negative_word(latest_ten_k)/len(latest_ten_k)
# final_df
# str_year = str(max_ten_k_year) + '-12-31'
# str_year
# # len(latest_ten_k)

{2015: 'cleaned_2015-02-24_10-K',
 2017: 'cleaned_2017-02-17_10-K',
 2018: 'cleaned_2018-02-16_10-K',
 2019: 'cleaned_2019-02-13_10-K',
 2014: 'cleaned_2014-02-25_10-K',
 2016: 'cleaned_2016-02-19_10-K'}

In [415]:
# Let's get next year outlooks

import yfinance as yf


tickers_dict = {'AMAZON COM INC': 'AMZN', 'APPLE INC': 'AAPL', 'AMERICAN EXPRESS CO':'AXP', 'MICROSOFT CORP': 'MSFT', 'Facebook Inc': 'META', 'NETFLIX INC': 'NFLX'}

tickers = ['AMZN', 'AAPL', 'AXP', 'MSFT', 'META', 'NFLX']  # Replace with the stock symbol you want

annual_returns = pd.DataFrame()
for ticker in tickers:
    data = yf.download(ticker, start="2000-01-01", end="2024-01-01", interval='1d')

    # Resample the data to get the price at the end of each year
    data_yearly = data['Adj Close'].resample('Y').last()


    # Calculate annual returns: (Price in current year - Price in previous year) / Price in previous year
    annual_returns[ticker] = data_yearly.pct_change()

# Drop rows with NaN values (usually the first year will have NaN)
# annual_returns = data[['Yearly Close', 'Annual Return']].dropna()
annual_returns = annual_returns.dropna()
annual_returns.index = annual_returns.index.year

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [464]:
# Let's get next year outlooks

import yfinance as yf


tickers_dict = {'AMAZON COM INC': 'AMZN', 'APPLE INC': 'AAPL', 'AMERICAN EXPRESS CO':'AXP', 'MICROSOFT CORP': 'MSFT', 'Facebook Inc': 'META', 'NETFLIX INC': 'NFLX'}

tickers = ['AMZN', 'AAPL', 'AXP', 'MSFT', 'META', 'NFLX']  # Replace with the stock symbol you want

annual_returns = pd.DataFrame()
for ticker in tickers:
    data = yf.download(ticker, start="2000-01-01", end="2024-01-01", interval='1d')

    # Resample the data to get the price at the end of each year
    data_yearly = data['Adj Close'].resample('M').last()


    # Calculate annual returns: (Price in current year - Price in previous year) / Price in previous year
    annual_returns[ticker] = data_yearly.pct_change()

# Drop rows with NaN values (usually the first year will have NaN)
# annual_returns = data[['Yearly Close', 'Annual Return']].dropna()
annual_returns = annual_returns.dropna()
annual_returns

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,AMZN,AAPL,AXP,MSFT,META,NFLX
Date,,,,,,
2012-06-30,0.072519,0.010853,0.042629,0.047962,0.050676,0.079603
2012-07-31,0.021677,0.045822,-0.005220,-0.036613,-0.301929,-0.169952
2012-08-31,0.064166,0.093876,0.010224,0.052737,-0.168125,0.050484
2012-09-30,0.024369,0.002796,-0.024700,-0.034393,0.199336,-0.088413
2012-10-31,-0.084264,-0.107600,-0.012198,-0.040994,-0.025392,0.455547
...,...,...,...,...,...,...
2023-08-31,0.032391,-0.042384,-0.064484,-0.022226,-0.071281,-0.012051
2023-09-30,-0.078907,-0.088678,-0.055700,-0.036643,0.014600,-0.129312
2023-10-31,0.046963,-0.002570,-0.017159,0.070815,0.003531,0.090281


In [465]:
final_df

,year,company,litigous_score,superfluous_score,interesting_score,modal_score,harvard_1,polarity_score,avg_sen_length,fog_index,Future Return
0,2014,AMERICAN EXPRESS CO,0.017580,0.000224,0.001607,0.014124,0.047509,0.053458,33202,13280.973893,0.036627
1,2015,AMERICAN EXPRESS CO,0.018550,0.000172,0.001603,0.014447,0.048322,0.051530,32329,12931.773862,-0.242095
2,2016,AMERICAN EXPRESS CO,0.013058,0.000177,0.001374,0.011426,0.053947,0.057735,46143,18457.367887,0.085822
3,2017,AMERICAN EXPRESS CO,0.013551,0.000149,0.001588,0.012558,0.057231,0.061721,37799,15119.770491,0.362164
4,2018,AMERICAN EXPRESS CO,0.012938,0.000081,0.001629,0.012615,0.058108,0.064072,38732,15492.970464,-0.026202
5,2019,AMERICAN EXPRESS CO,0.013071,0.000118,0.001368,0.013307,0.056927,0.065217,36912,14764.973992,0.325174
6,2014,Facebook Inc,0.010787,0.000109,0.001775,0.017205,0.042083,0.045054,21236,8494.584065,0.427630
7,2015,Facebook Inc,0.010144,0.000141,0.001803,0.017583,0.044043,0.044256,20623,8249.385967,0.341451
8,2016,Facebook Inc,0.010719,0.000114,0.001686,0.018550,0.045590,0.046100,20292,8116.984073,0.099274
9,2017,Facebook Inc,0.012631,0.000104,0.001641,0.019142,0.046903,0.047337,22332,8932.981426,0.533768


In [457]:
annual_returns
for i in range(len(final_df)):
    final_df.loc[i, 'Future Return'] = annual_returns.loc[final_df.loc[i,'year']][tickers_dict[final_df.loc[i, 'company']]]

In [436]:
# final_df.to_csv('/Users/rishikumra/Downloads/Projects/sentiment-analysis-sec-master/data/cleaned_data.csv')

In [459]:
final_df.to_csv('/Users/rishikumra/Downloads/Projects/sentiment-analysis-sec-master/data/data_2cleaned.csv')